# 分布式Tensorflow训练任务

您可以在Notebook中使用TI SDK来提交训练任务。TI SDK不仅支持Tensorflow，PyTorch等流行的机器学习框架，还允许用户使用自定义镜像。这个示例展示如何使用TI SDK来提交分布式版本的Tensorflow任务

#### 引入依赖

In [ ]:
from __future__ import absolute_import, print_function

import sys
from ti import session
from ti.tensorflow import TensorFlow

#### 准备训练数据和脚本

我们已经在对象存储COS中准备好了一份训练数据(点击[此链接](https://ti-ap-guangzhou-1300268737.cos.ap-guangzhou.myqcloud.com/training_data/tensorflow/mnist/mnist.npz)下载)，您也可以替换为自己的数据。

提交一个任务的步骤包括：
1. 准备训练数据
2. 创建一个Tensorflow Estimator，这里可以指定各种需要的参数
3. 调用fit()提交训练任务

TI提供基于Horovod的分布式训练框架，通过参数_distributions_来启动一个Tensorflow分布式训练任务

mpi: 使用Horovod框架，支持的参数有
- enabled: 标识使用Horovod，默认True
- processes_per_host: 每个节点的进程数
- custom_mpi_options: 自定义参数

In [ ]:
# 初始化一个session
ti_session = session.Session()

# 授权给TI的服务角色
role = "TIONE_QCSRole"

# COS中的训练数据
inputs = 'cos://ti-ap-guangzhou-1300268737/training_data/tensorflow/mnist/'

# 设置算法超级参数
hyperparameters = {'epochs': 30}

# 创建一个Tensorflow Estimator
tf_estimator = TensorFlow(role=role,
                          train_instance_count=2,
                          train_instance_type='TI.GN10X.2XLARGE40.1xV100',
                          py_version='py3',
                          script_mode=True,
                          hyperparameters=hyperparameters,
                          framework_version='1.14.0',
                          entry_point='train.py',
                          source_dir='horovod',
                          image_name="ccr.ccs.tencentyun.com/ti_public/tensorflow:1.14.0-py3-dist",
                          distributions={
                              'mpi': {
                                  'enabled': True,
                                  'processes_per_host': 1,
                                  'custom_mpi_options': '--NCCL_DEBUG INFO'
                              }
                          })
# 提交Tensorflow训练任务
tf_estimator.fit(inputs)

## 查看输出模型


如果训练任务完成有模型输出，那么TI会将模型上传到cos上。
你可以通过**output_path='cos://'**指定模型存储的cos路径。如果没指定，TI会按以下格式创建存储通
**ti-[region]-[uin]-[appid]**

最终模型会放在 #{bucket}/#{job_name}/output/目录下


## 查看训练日志

TI训练过程中会把日志上传到[腾讯云日志服务](https://cloud.tencent.com/product/cls)中，使用腾讯云日志服务需要先开通。

TI会针对训练任务创建TiOne日志集和TrainingJob日志主题，通过日志检索功能可以搜索对应训练任务的日志。


目前TI会默认创建一个日志集(TI)和日志主题(TrainingJob)

TI内置了任务名称(job)关键词，可以通过以下条件过滤指定任务的日志

```
job: #{job_name}
```

更多日志检索语法请参考https://cloud.tencent.com/document/product/614/16981

## 查看提交历史和监控

您可以在TI产品控制台上的【任务列表】-【SDK任务】中查看您提交的SDK任务历史，点击【任务名称】列中的任务名可以看到任务的详细信息，在列表的右侧【监控】可以查看任务运行的资源监控图表



## 更多的TI SDK使用说明可以参考官网智能钛产品文档